# Keep-it-dry! LogisticRegression x AdaBoost
Dataset: ki_ro_ros.csv

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier

from pprint import pprint

## Data Read

In [3]:
df_train = pd.read_csv('./../Preprocessing/kid_train_ki_ro_ros.csv', index_col=0)
df_test = pd.read_csv('./../Preprocessing/kid_test_ki_ro.csv', index_col=0)

In [4]:
df_train.head()

,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,-0.866789,7.0,8.0,9,5,0.000000,0.000000,-0.50,0.191386,0.587838,...,-0.582308,-0.155224,-0.823018,1.817328,-0.416608,-0.826421,-0.996926,-0.798281,0.405153,0.0
1,-0.768774,7.0,8.0,9,5,1.166667,-0.833333,-0.75,0.320974,-0.146396,...,0.783846,1.031983,-0.658824,0.022965,-0.627395,-0.310259,-0.299693,-0.369968,-0.127292,0.0
2,-0.819112,7.0,8.0,9,5,0.833333,-1.166667,-0.25,0.204120,-0.062312,...,0.989231,-0.298507,-0.298414,1.089248,0.777147,1.355888,-0.453893,0.677069,-0.248529,0.0
3,-0.437692,7.0,8.0,9,5,1.000000,-1.000000,0.00,-0.366667,-0.410661,...,0.801538,0.121677,-0.422506,-0.882568,-0.259759,-0.239060,0.601434,0.327001,0.808704,0.0
4,1.342337,7.0,8.0,9,5,0.333333,-1.000000,0.50,1.169663,0.912162,...,-0.840000,0.540156,0.372890,0.374217,0.401703,-1.695250,-0.935963,-0.016735,-0.790371,0.0


In [5]:
df_test_id = df_test[['id']]
df_test.drop(columns=['id', 'product_code', 'product_code_F', 'product_code_G', 'product_code_H', 'product_code_I'], inplace=True)
df_test.head()

,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,-0.059136,5.0,6.0,6,4,-0.166667,0.166667,0.0,1.138951,-1.168919,...,-0.274792,-0.482308,-0.126795,-0.579540,1.075678,-1.388928,0.417316,-0.634221,0.570330,-0.435202
1,-0.183139,5.0,6.0,6,4,0.666667,0.000000,-1.5,0.073783,0.144144,...,0.265708,0.463846,-1.213362,-0.441432,0.397704,1.314407,0.356512,-0.104508,-1.061511,-1.068447
2,-0.210763,5.0,6.0,6,4,0.166667,0.666667,-0.5,0.517228,-0.941441,...,-0.940954,0.241538,0.519687,-0.571355,-0.419624,-1.599006,-0.148114,1.068135,-0.193578,-0.276961
3,-0.199304,5.0,6.0,6,4,0.166667,0.500000,1.0,-0.948689,-0.635886,...,-0.052233,0.277692,1.155935,-1.554987,-0.406054,-0.081618,-0.184492,-1.209016,-0.152872,-0.696814
4,1.750358,5.0,6.0,6,4,1.166667,1.333333,0.5,0.017603,0.719219,...,0.093868,0.723846,-0.883582,-0.692583,0.120042,0.322214,0.083671,-0.848361,0.317051,0.644913


In [6]:
# Scale
# Scaled already

## Hyperparameter Tuning using GridSearchCV

In [7]:
X_df = df_train[df_train.columns[:-1]]
y_df = df_train[df_train.columns[-1]]

X_train, X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=.2, random_state=42, stratify=y_df)
X_train.head()

,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
26505,-0.137098,7.0,6.0,6,9,-0.500000,-0.666667,-0.75,-1.635581,0.207207,...,0.115821,-0.757692,-1.609097,1.456266,-0.755219,-0.580554,0.805530,-0.320184,-0.984170,0.297649
5022,0.609986,7.0,8.0,9,5,0.000000,-0.666667,0.00,0.237828,1.216967,...,0.467070,-0.183846,-0.208102,1.370844,0.374217,-0.046842,1.390708,0.679816,-1.108096,1.502298
8546,-0.345611,5.0,5.0,8,8,-0.333333,-1.333333,1.75,-0.306742,0.066066,...,1.037850,-0.144615,-0.560057,-0.732379,-0.357516,-1.381831,-0.018189,-1.765369,0.819539,-0.009359
8107,-0.542664,5.0,5.0,8,8,-0.500000,0.333333,0.25,1.194382,-0.167417,...,1.464799,0.903077,0.507178,-0.109974,-0.186117,1.340667,-0.992620,-0.091701,1.065129,0.671093
6654,0.553100,5.0,5.0,8,8,-0.333333,-0.333333,-0.25,0.559925,-0.289790,...,1.305072,0.147692,0.183653,-0.572379,-0.521399,0.660043,0.055088,1.388320,0.828584,0.020747


In [8]:
# Define model
model = LogisticRegression(max_iter=1000)

# Define hyperparameter grid
param_grid = {
    'solver': ['liblinear', 'lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    'penalty': ['l2', 'elasticnet'],
    'C': np.logspace(-2, 2, 10)  # Search across different C values in log space
}

# Define GridSearchCV
kfolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kfolds, scoring='f1')  # Use F1 score for imbalanced data

In [9]:
# GridSearchCV fitting
grid_search.fit(X_train, y_train)

c:\Users\RAYHAN EGAR\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
600 fits failed out of a total of 1200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\RAYHAN EGAR\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\RAYHAN EGAR\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\RAYHAN EGAR\anaconda3\Lib\site-packa

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02]),
                         'penalty': ['l2', 'elasticnet'],
                         'solver': ['liblinear', 'lbfgs', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1')

In [11]:
logreg_best = grid_search.best_estimator_
logreg_best_params = grid_search.best_params_
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best f1-score: {grid_search.best_score_}")

Best parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}
Best f1-score: 0.5412149752015489


## AdaBoost

In [25]:
boosting_clf = AdaBoostClassifier(base_estimator=LogisticRegression(**logreg_best_params), n_estimators=20)
boosting_clf.fit(X_train, y_train)

c:\Users\RAYHAN EGAR\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(base_estimator=LogisticRegression(C=0.01,
                                                     solver='liblinear'),
                   n_estimators=20)

In [26]:
adaboost_f1 = cross_val_score(boosting_clf, X_train, y_train, cv=kfolds, scoring='f1')

print("F1 Scores:", adaboost_f1)
print("Average F1 Score:", np.mean(adaboost_f1))
print("Standard Deviation:", np.std(adaboost_f1))

c:\Users\RAYHAN EGAR\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\RAYHAN EGAR\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\RAYHAN EGAR\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\RAYHAN EGAR\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\RAYHAN EGAR\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Us

F1 Scores: [0.55611789 0.57326704 0.54898447 0.55129349 0.57454331 0.54007177
 0.56384248 0.54205052 0.55386452 0.57293807]
Average F1 Score: 0.5576973556629298
Standard Deviation: 0.012189491900656114


## Prediction

In [27]:
y_pred = boosting_clf.predict(df_test)
y_pred_df = pd.DataFrame(y_pred, columns=['failure'])

In [28]:
y_submission = pd.concat([df_test_id, y_pred_df], axis=1)

In [29]:
y_submission.head()

,id,failure
0,26570,1.0
1,26571,0.0
2,26572,0.0
3,26573,1.0
4,26574,1.0


In [30]:
y_submission.to_csv('./../Submission/kid_submission_logreg_adaboost_v2.csv', index=False)